In [4]:
import numpy as np
import matplotlib.pyplot as plt
from scipy.linalg import matmul_toeplitz

from ppfft.tools.grids import domain

# Fast frac fft

The frac fft we defined here is actually slow due to zero-padding to size $n^2$ during the ppfft. Here, we define a general frac fft:
$$
\forall u \in \mathcal{D}(n), X(u) = \sum_{j \in \mathcal{D}(n)} x(j) \exp\left(-2i\pi \beta j u \right)
$$

In [5]:
def frac_fft(x, beta, m=None):
    n = len(x)
    w = np.exp(-2j * np.pi * beta)
    # Output size is the same as input.
    if m is None:
        dom = domain(n)
        w_powers = w ** (0.5 * dom ** 2) # this may not be optimal
        c = w ** (- 0.5 * (dom - dom[0]) ** 2)
        return w_powers * matmul_toeplitz((c, c), w_powers * x)
    # Output size given by m.
    else:
        dom_n = domain(n)
        dom_m = domain(m)
        w_powers_n = w ** (0.5 * dom_n ** 2) # this may not be optimal
        w_powers_m = w ** (0.5 * dom_m ** 2) # this may not be optimal
        c = w ** (- 0.5 * (dom_m - dom_n[0]) ** 2)
        r = w ** (- 0.5 * (dom_n - dom_m[0]) ** 2)
        return w_powers_m * matmul_toeplitz((c, r), w_powers_n * x)

In [6]:
def test_fast_frac_fft(x, beta, m=None):

    n = len(x)
    if m is None:
        us = domain(n)
    else:
        us = domain(m)
    
    js = domain(n)

    res = []
    for u in us:
        res.append(np.sum(x * np.exp(-2j * np.pi * beta * u * js)))

    return np.array(res)

In [7]:
n = 1000
x = np.random.rand(n)
beta = np.random.rand()
np.allclose(frac_fft(x, beta), test_fast_frac_fft(x, beta))

True

In [8]:
n = 1001
x = np.random.rand(n)
beta = np.random.rand()
np.allclose(frac_fft(x, beta), test_fast_frac_fft(x, beta))

True

In [9]:
n = 1000
x = np.random.rand(n)
beta = np.random.rand()
np.allclose(frac_fft(x, beta, m=n+1), test_fast_frac_fft(x, beta, m=n+1))

True

In [10]:
n = 1001
x = np.random.rand(n)
beta = np.random.rand()
np.allclose(frac_fft(x, beta, m=n+1), test_fast_frac_fft(x, beta, m=n+1))

True

Adjoint:

In [11]:
def adj_frac_fft(y, beta, n):
    return frac_fft(y, -beta, m=n)

In [14]:
n = 10
m = n + 1
x = np.random.rand(n)
y = np.random.rand(m)
beta = np.random.rand()

np.isclose(np.vdot(frac_fft(x, beta, m), y), np.vdot(x, adj_frac_fft(y, beta, n)))

True

In [15]:
n = 11
m = n + 1
x = np.random.rand(n)
y = np.random.rand(m)
beta = np.random.rand()

np.isclose(np.vdot(frac_fft(x, beta, m), y), np.vdot(x, adj_frac_fft(y, beta, n)))

True